In [3]:
import sys
import qgrid
import copy

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

sys.path.append('../../lib/')
from dftmanlib.pwscf import pwcalculation_helper, pseudo_helper
from dftmanlib.job import PBSJob
from dftmanlib.matproj import mpquery_helper
from dftmanlib.db import load_db

from tinydb import Query

qgrid.enable()
db = load_db()
table = db.table('SubmitJob')

PSEUDO_TABLE = '/data/tools/dftman/pseudo_table.json'
# available pseudo families:
# ['SSSP_EFFICIENCY', 'SSSP_PRECISION', 'GBRV_US_LDA',
#  'GBRV_US_PBE', 'GBRV_US_PBEsol', 'DOJO_STANDARD_LDA_NC',
#  'DOJO_STANDARD_PBE_NC', 'DOJO_STANDARD_PBEsol_NC',
#  'DOJO_STRINGENT_LDA_NC', 'DOJO_STRINGENT_PBE_NC',
#  'DOJO_STRINGENT_PBAsol_NC']
PSEUDO_FAMILY = 'GBRV_US_PBE'
MP_API_KEY = '0WqdPfXxloze6T9N'

## Materials Project Query

In [4]:
criteria = {
    'elements': 'Al',
    'nsites': 1,
    'spacegroup.number': 225,
}
properties = []
m = mpquery_helper(criteria, properties, MP_API_KEY)
m.query()
m.display()

QgridWidget(grid_options={'fullWidthRows': True, 'syncColumnCellResize': True, 'forceFitColumns': True, 'defaultColumnWidth': 150, 'rowHeight': 28, 'enableColumnReorder': False, 'enableTextSelectionOnCells': True, 'editable': True, 'autoEdit': False, 'explicitInitialization': True, 'maxVisibleRows': 15, 'minVisibleRows': 8, 'sortable': True, 'filterable': True, 'highlightSelectedCell': False, 'highlightSelectedRow': True}, id='6e644f0a-b378-42e6-b54c-af257e3dff2b', precision=5)

### Base configuration

In [5]:
STRUCTURE = m.result[0]['structure']

PSEUDO = pseudo_helper(STRUCTURE, PSEUDO_FAMILY,
                       PSEUDO_TABLE)
base_inputs = {
        'structure': STRUCTURE,
        'pseudo': PSEUDO,

        'control': {
            'calculation': 'scf',
            'verbosity': 'high',
            'disk_io': 'none',
        },
        'system': {
            'ibrav': 0,
            'ecutwfc': 45,
            'occupations': 'smearing',
            'degauss': 0.01,
            'smearing': 'mv',
        },
        'electrons': {
            'electron_maxstep': 500,
            'conv_thr': 1.0e-6,
        },
        'ions': {},
        'cell': {},
        'kpoints_mode': 'automatic',
        'kpoints_grid': (20, 20, 20),
        'kpoints_shift': (0, 0, 0),
    }

### Customize, store, and run individual calculations

In [6]:
calculation = pwcalculation_helper(**base_inputs)
command = 'pw.x < {input_path:s} > {output_path:s}'

In [7]:
job = PBSJob(calculation, command, walltime='01:00:00', nnodes=1, ppn=20, np=20, queue='standby',
             runname='test1')

In [8]:
job.write_input()
job.write_script()

In [9]:
!cat jobs/test1_9a3944622d6f/pbs_runscript.sh


#!/bin/bash
#PBS -l walltime=01:00:00
#PBS -l nodes=1:ppn=20
#PBS -q standby
#PBS -N test1



mpirun -np 20 mpirun -np 20 pw.x < /home/nanohub/azadoks/git/dftman/projects/PBS/jobs/test1_9a3944622d6f/pwscf.in > /home/nanohub/azadoks/git/dftman/projects/PBS/jobs/test1_9a3944622d6f/pwscf.out


